In [1]:
import h5py
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

from utils import (coupling_coef_corrs,
                   selection_profiles_by_chance,
                   get_dataset,
                   create_graph)
from pyuoi.linear_model import UoI_Lasso
from sklearn.metrics import r2_score
%matplotlib inline

In [2]:
fits_path = '/Users/psachdeva/fits/uoineuro/pvc11/pvc11_monkey1.h5'

In [3]:
fits = h5py.File(fits_path, 'r')

In [4]:
Y1 = fits['UoI_Lasso_r2/Y'][:]
Y2 = fits['Lasso/Y'][:]
assert np.all(Y1 == Y2)
Y = Y1

In [11]:
neuron_idx = 3
fold_idx = 4
train_fold = fits['UoI_Lasso_r2/training_folds/fold_%s' %fold_idx]
test_fold = fits['UoI_Lasso_r2/test_folds/fold_%s' %fold_idx]
coupling_coefs = fits['UoI_Lasso_r2/coupling_coefs'][fold_idx, neuron_idx]
intercept = fits['UoI_Lasso_r2/intercepts'][fold_idx, neuron_idx]
X_train = np.delete(Y, neuron_idx, axis=1)[train_fold, :]
X_test = np.delete(Y, neuron_idx, axis=1)[test_fold, :]
y_train = Y[:, neuron_idx][train_fold]
y_test = Y[:, neuron_idx][test_fold]

In [12]:
print(r2_score(y_train, intercept + np.dot(X_train, coupling_coefs)))
print(fits['UoI_Lasso_r2/r2'][fold_idx, neuron_idx])

-5.581682019336263
-5.4941700562787865


In [13]:
fitter = UoI_Lasso(
    n_boots_sel=30,
    n_boots_est=30,
    selection_frac=0.8,
    estimation_frac=0.8,
    estimation_score='r2',
    stability_selection=1.0,
    standardize=True,
    fit_intercept=True,
    max_iter=5000
)

In [14]:
fitter.fit(X_train, y_train)

UoI_Lasso(comm=None, copy_X=None, eps=0.001, estimation_frac=0.8,
          estimation_score='r2', fit_intercept=True, max_iter=5000,
          n_boots_est=30, n_boots_sel=30, n_lambdas=48,
          random_state=<module 'numpy.random' from '/Users/psachdeva/anaconda3/lib/python3.6/site-packages/numpy/random/__init__.py'>,
          selection_frac=0.8, stability_selection=1.0, standardize=True,
          warm_start=None)

In [15]:
fitter.intercept_

array([0.31460171])

In [16]:
r2_score(y_test, np.dot(X_test, fitter.coef_) + fitter.intercept_)

0.753391292822865

In [17]:
fitter.scores_

array([[-2.20563670e-03,  6.40540460e-01,  6.74245806e-01, ...,
         7.28733867e-01,  7.28610952e-01,  7.28742354e-01],
       [-1.63539297e-03,  6.55210441e-01,  6.93117941e-01, ...,
         7.70753343e-01,  7.70954360e-01,  7.70503911e-01],
       [-3.42336769e-04,  6.61751497e-01,  7.19205886e-01, ...,
         7.92679240e-01,  7.92884077e-01,  7.92834220e-01],
       ...,
       [-3.46616532e-05,  6.72693749e-01,  7.36214471e-01, ...,
         7.83309025e-01,  7.83689151e-01,  7.83843980e-01],
       [-7.48494147e-03,  6.04918613e-01,  6.55875540e-01, ...,
         7.39577353e-01,  7.38724924e-01,  7.37754456e-01],
       [-5.09598657e-04,  6.84111694e-01,  7.42569951e-01, ...,
         8.02961584e-01,  8.03117440e-01,  8.03216098e-01]])

In [60]:
from sklearn.linear_model import LassoCV
lasso = LassoCV(normalize=True)
lasso.fit(X_train, y_train)
print(lasso.intercept_)

/Users/psachdeva/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


0.2140267073165072
